In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os
import tensorflow as tf
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
 #       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
import tensorflow as tf
def _float_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))
def _byte_feature(value):
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=[value]))
def _int_64_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))
def serialize_array(value):
    return tf.io.serialize_tensor(value)


In [3]:
def makeEx(image_filename, label):
    image = np.asarray(Image.open(image_filename).resize((400,400)))
    data = {
        "height": _int_64_feature(image.shape[0]),
        "width" : _int_64_feature(image.shape[1]),
        "channels": _int_64_feature(image.shape[2]),
        "label": _byte_feature(bytes(label)),
        "image": _byte_feature(serialize_array(image)),
    }
    
    return tf.train.Example(features=tf.train.Features(feature=data))
def makeRecordFile(files, labels, filename):
    filename += ".tfrecords"
    writer = tf.io.TFRecordWriter(filename)
    count = 0
    for i in range(len(files)):
        example = makeEx(files[i], bytes(labels[i],'utf-8'))
        writer.write(example.SerializeToString())
        count += 1
    
    writer.close()
    print("Wrote {} to the record file".format(count))


In [4]:
from PIL import Image
import os
i = 0
os.chdir("/kaggle/input/flickr8kimagescaptions/flickr8k/images")
files = os.listdir(os.getcwd())
labels = []
with open("/kaggle/input/flickr8kimagescaptions/flickr8k/captions.txt", "r") as f:
    labels = f.read().split("\n")[1:]
#for file in files:
 #   np.asarray(Image.open(file))
  #  file.

In [5]:
labels = [i[i.find(".jpg")+5:] for i in labels]

In [6]:
#https://towardsdatascience.com/a-practical-guide-to-tfrecords-584536bc786c
labels
unique_characters = list(set("".join(labels)))
num_unique = len(unique_characters)
print("The unique characters are " + "".join(unique_characters) + " and number " + str(num_unique))

The unique characters are bdBsx7TS4vetWl.kV;9zY"q03hDOGaNuU18rMc:P(pKFHy#I6Q,)!o?jg-R5Z2=ifACm w'ELJn&X and number 77


In [7]:
makeRecordFile(files, labels, "/kaggle/working/hello")

Wrote 8091 to the record file
